#### Lisa Over
CMPINF 2110<br>
Homework 3: Tuberculosis Data<br>
February 24, 2021

In [404]:
import sys
print(sys.executable)

/Users/lisa/opt/anaconda3/envs/cmpinf_2110/bin/python


In [405]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

The Tuberculosis (TB) data contains information that is redundant and not necessary to have in the data file when considering storage. The columns country, sex, and age_group contain values that are short strings, no more than five characters each. These values could stay in the data file because they do not take up much storage space, however, this information could also be enhanced with additonal details and columns. This additional information would produce more redundant and unnecessary data in the stored TB data file.

For the country data, I created a country_info dataframe using unique values from the country column. I then added country name so country_info consists of both the country (ISO ALPHA-2 code) and country_name. The country column has some NaN values so I filled those with 'XU' and added a row to the country_info column with country_name='Unknown' and country='XU'. Finally, I created a country_id from the index, merged the TB data with the country_info to add the country_id to the TB data, and then removed the redundant country and country_name columns from the TB data. 

For the sex data, I created a sex_info dataframe using unique values from the sex column. I then added sex_name, sex_iso (ISO sex codes), and sex_who (WHO sex codes) columns. Finally, I created a sex_id using the ISO standard, merged the TB data with sex_info to add the sex_id to the TB data, and finally removed the redundant sex, sex_iso, sex_who, and sex_name columns from the TB data.

I could not find ISO or WHO codes to add to the age_group data. I found all age groups in the WHO codes except 65+ (instead they used 65-100 and 100+). I still created a table for age_group_info so more information could be added later. The age_group column has some NaN values so I filled those with 'unknown'. I created the age_group_info dataframe using unique values from the age_group column and added a row where age_group='unknown'. Finally, I created an age_group_id, merged the TB data with age_group_info to add the age_group_id to the TB data, and then removed the redundant age_group column. 

#### Read and display the data

In [406]:
file_name = "tb_tidy.csv"
tb_df = pd.read_csv(file_name)
tb_df.head()

,Unnamed: 0,country,year,value,sex,age_group
0,0,AD,1989,NaN,f,0-14
1,1,AD,1990,NaN,f,0-14
2,2,AD,1991,NaN,f,0-14
3,3,AD,1992,NaN,f,0-14
4,4,AD,1993,NaN,f,0-14


In [407]:
tb_df.year.value_counts()

2008    3392
2006    3392
2007    3392
2002    3328
2003    3328
2004    3312
2005    3296
2001    3280
1998    3248
1988    3216
1999    3216
2000    3216
1996    3200
1987    3184
1997    3168
1985    3168
1989    3152
1986    3152
1995    3152
1983    3136
1990    3136
1981    3104
1982    3104
1984    3088
1991    3072
1980    3056
1992    2992
1994    2944
1993    2880
Name: year, dtype: int64

#### Check rows with missing 'value' column values

In [408]:
tb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92304 entries, 0 to 92303
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  92304 non-null  int64  
 1   country     91936 non-null  object 
 2   year        92304 non-null  int64  
 3   value       34160 non-null  float64
 4   sex         92304 non-null  object 
 5   age_group   80766 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 4.2+ MB


In [409]:
tb_df['value'].isnull().sum()

58144

In [410]:
missing_value = tb_df.loc[tb_df['value'].isna(), :]
missing_value

,Unnamed: 0,country,year,value,sex,age_group
0,0,AD,1989,NaN,f,0-14
1,1,AD,1990,NaN,f,0-14
2,2,AD,1991,NaN,f,0-14
3,3,AD,1992,NaN,f,0-14
4,4,AD,1993,NaN,f,0-14
...,...,...,...,...,...,...
92298,92298,NaN,2003,NaN,m,NaN
92299,92299,NaN,2004,NaN,m,NaN
92300,92300,NaN,2005,NaN,m,NaN
92301,92301,NaN,2006,NaN,m,NaN


#### Check rows with missing 'country' column values

In [411]:
tb_df['country'].isnull().sum()

368

In [412]:
missing_country = tb_df.loc[tb_df['country'].isna(), :]
missing_country

,Unnamed: 0,country,year,value,sex,age_group
5746,5746,NaN,1985,NaN,f,0-14
5747,5747,NaN,1986,NaN,f,0-14
5748,5748,NaN,1987,NaN,f,0-14
5749,5749,NaN,1988,NaN,f,0-14
5750,5750,NaN,1989,NaN,f,0-14
...,...,...,...,...,...,...
92299,92299,NaN,2004,NaN,m,NaN
92300,92300,NaN,2005,NaN,m,NaN
92301,92301,NaN,2006,NaN,m,NaN
92302,92302,NaN,2007,NaN,m,NaN


In [413]:
missing_country.groupby(["year"]).size().sort_values(ascending=False)

year
1985    16
1998    16
2007    16
2006    16
2005    16
2004    16
2003    16
2002    16
2001    16
2000    16
1999    16
1997    16
1986    16
1996    16
1995    16
1993    16
1992    16
1991    16
1990    16
1989    16
1988    16
1987    16
2008    16
dtype: int64

#### Fill NaN country values with 'XU' for unknown

Set NaN values to 'XU' for Unknown, see  https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2 "User-assigned code elements."

In [414]:
tb_df['country'] = tb_df['country'].fillna('XU');
tb_df

,Unnamed: 0,country,year,value,sex,age_group
0,0,AD,1989,NaN,f,0-14
1,1,AD,1990,NaN,f,0-14
2,2,AD,1991,NaN,f,0-14
3,3,AD,1992,NaN,f,0-14
4,4,AD,1993,NaN,f,0-14
...,...,...,...,...,...,...
92299,92299,XU,2004,NaN,m,NaN
92300,92300,XU,2005,NaN,m,NaN
92301,92301,XU,2006,NaN,m,NaN
92302,92302,XU,2007,NaN,m,NaN


#### Get list of country names and merge with tb_cdata

In [415]:
tb_cdata = tb_df.copy()
tb_cdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92304 entries, 0 to 92303
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  92304 non-null  int64  
 1   country     92304 non-null  object 
 2   year        92304 non-null  int64  
 3   value       34160 non-null  float64
 4   sex         92304 non-null  object 
 5   age_group   80766 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 4.2+ MB


In [416]:
tb_cdata.country.value_counts()

JM    464
TO    464
SI    464
CA    464
DK    464
     ... 
AN    208
VG    160
TL    128
ME     64
RS     64
Name: country, Length: 213, dtype: int64

Get list of country names and codes from https://datahub.io/core/country-list#data.

In [417]:
data_url = 'https://pkgstore.datahub.io/core/country-list/data_csv/data/d7c9d7cfb42cb69f4422dec222dbbaa8/data_csv.csv'
country_codes = pd.read_csv(data_url)
country_codes.head()

,Name,Code
0,Afghanistan,AF
1,Åland Islands,AX
2,Albania,AL
3,Algeria,DZ
4,American Samoa,AS


In [418]:
country_codes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    249 non-null    object
 1   Code    248 non-null    object
dtypes: object(2)
memory usage: 4.0+ KB


In [419]:
missing_code = country_codes.loc[country_codes['Code'].isna(), :]
missing_code.head()

,Name,Code
153,Namibia,NaN


The row with missing code is not really missing. For some reason, Python interpreted 'NA' as NaN. Resolve this by assiging 'NA' to Namibia.

In [420]:
country_codes.loc[country_codes['Name'] == 'Namibia', 'Code'] = 'NA'

In [421]:
country_codes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    249 non-null    object
 1   Code    249 non-null    object
dtypes: object(2)
memory usage: 4.0+ KB


Create a list of codes to compare with codes in tb_cdata.

In [422]:
codes = country_codes.Code.to_list()
codes.sort()
codes

['AD',
 'AE',
 'AF',
 'AG',
 'AI',
 'AL',
 'AM',
 'AO',
 'AQ',
 'AR',
 'AS',
 'AT',
 'AU',
 'AW',
 'AX',
 'AZ',
 'BA',
 'BB',
 'BD',
 'BE',
 'BF',
 'BG',
 'BH',
 'BI',
 'BJ',
 'BL',
 'BM',
 'BN',
 'BO',
 'BQ',
 'BR',
 'BS',
 'BT',
 'BV',
 'BW',
 'BY',
 'BZ',
 'CA',
 'CC',
 'CD',
 'CF',
 'CG',
 'CH',
 'CI',
 'CK',
 'CL',
 'CM',
 'CN',
 'CO',
 'CR',
 'CU',
 'CV',
 'CW',
 'CX',
 'CY',
 'CZ',
 'DE',
 'DJ',
 'DK',
 'DM',
 'DO',
 'DZ',
 'EC',
 'EE',
 'EG',
 'EH',
 'ER',
 'ES',
 'ET',
 'FI',
 'FJ',
 'FK',
 'FM',
 'FO',
 'FR',
 'GA',
 'GB',
 'GD',
 'GE',
 'GF',
 'GG',
 'GH',
 'GI',
 'GL',
 'GM',
 'GN',
 'GP',
 'GQ',
 'GR',
 'GS',
 'GT',
 'GU',
 'GW',
 'GY',
 'HK',
 'HM',
 'HN',
 'HR',
 'HT',
 'HU',
 'ID',
 'IE',
 'IL',
 'IM',
 'IN',
 'IO',
 'IQ',
 'IR',
 'IS',
 'IT',
 'JE',
 'JM',
 'JO',
 'JP',
 'KE',
 'KG',
 'KH',
 'KI',
 'KM',
 'KN',
 'KP',
 'KR',
 'KW',
 'KY',
 'KZ',
 'LA',
 'LB',
 'LC',
 'LI',
 'LK',
 'LR',
 'LS',
 'LT',
 'LU',
 'LV',
 'LY',
 'MA',
 'MC',
 'MD',
 'ME',
 'MF',
 'MG',
 'MH',

List codes in tb_cdata file that are not in the country_codes file.

In [423]:
tb_cdata.loc[~(tb_cdata.country.isin(codes))].country.unique()

array(['AN', 'YU', 'XU'], dtype=object)

Add 'YU,' 'AN', and 'XU' to country_codes dataset: 'YU' represents Yugoslavia and 'AN represents Netherlands Antilles per https://countrycode.org/netherlandsantilles. 'XU' represents unknown.

In [424]:
new_row = {'Name':'Yugoslavia', 'Code':'YU'}
country_codes = country_codes.append(new_row, ignore_index=True)
country_codes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    250 non-null    object
 1   Code    250 non-null    object
dtypes: object(2)
memory usage: 4.0+ KB


In [425]:
new_row = {'Name':'Netherlands Antilles', 'Code':'AN'}
country_codes = country_codes.append(new_row, ignore_index=True)
country_codes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    251 non-null    object
 1   Code    251 non-null    object
dtypes: object(2)
memory usage: 4.0+ KB


In [426]:
new_row = {'Name':'Unknown','Code':'XU'}
country_codes = country_codes.append(new_row, ignore_index=True)
country_codes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    252 non-null    object
 1   Code    252 non-null    object
dtypes: object(2)
memory usage: 4.1+ KB


#### Create country_info dataset

Rename country_codes columns, get tb countries from tb_cdata, create country_id, and merge with country_codes to get country_name into the country_info file.

In [427]:
country_codes.rename(columns={"Name": "country_name", "Code": "country"}, inplace=True)
country_codes.head()

,country_name,country
0,Afghanistan,AF
1,Åland Islands,AX
2,Albania,AL
3,Algeria,DZ
4,American Samoa,AS


In [428]:
tb_countries = tb_cdata.groupby(['country']).size().reset_index(name='num_rows')
tb_countries

,country,num_rows
0,AD,304
1,AE,416
2,AF,384
3,AG,432
4,AI,352
...,...,...
208,YE,336
209,YU,400
210,ZA,464
211,ZM,432


In [429]:
tb_countries.drop(['num_rows'], axis=1, inplace=True)
tb_countries

,country
0,AD
1,AE
2,AF
3,AG
4,AI
...,...
208,YE
209,YU
210,ZA
211,ZM


In [430]:
new_row = {'country':np.nan}
tb_countries = tb_countries.append(new_row, ignore_index=True)
tb_countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   country  213 non-null    object
dtypes: object(1)
memory usage: 1.8+ KB


In [431]:
country_info = pd.merge(country_codes, 
                        tb_countries,
                        how="right",
                        on="country")
country_info.head()

,country_name,country
0,Andorra,AD
1,United Arab Emirates,AE
2,Afghanistan,AF
3,Antigua and Barbuda,AG
4,Anguilla,AI


In [432]:
country_info['country_id'] = country_info.index + 1
country_info

,country_name,country,country_id
0,Andorra,AD,1
1,United Arab Emirates,AE,2
2,Afghanistan,AF,3
3,Antigua and Barbuda,AG,4
4,Anguilla,AI,5
...,...,...,...
209,Yugoslavia,YU,210
210,South Africa,ZA,211
211,Zambia,ZM,212
212,Zimbabwe,ZW,213


In [433]:
country_info.loc[country_info.country_name == 'Unknown', : ]

,country_name,country,country_id
207,Unknown,XU,208


In [434]:
country_info = country_info[['country_id','country','country_name']].copy()
country_info

,country_id,country,country_name
0,1,AD,Andorra
1,2,AE,United Arab Emirates
2,3,AF,Afghanistan
3,4,AG,Antigua and Barbuda
4,5,AI,Anguilla
...,...,...,...
209,210,YU,Yugoslavia
210,211,ZA,South Africa
211,212,ZM,Zambia
212,213,ZW,Zimbabwe


In [435]:
tb_plus_country = pd.merge(tb_cdata, 
                           country_info,
                           how="left",
                           on="country")
tb_plus_country.head()

,Unnamed: 0,country,year,value,sex,age_group,country_id,country_name
0,0,AD,1989,NaN,f,0-14,1,Andorra
1,1,AD,1990,NaN,f,0-14,1,Andorra
2,2,AD,1991,NaN,f,0-14,1,Andorra
3,3,AD,1992,NaN,f,0-14,1,Andorra
4,4,AD,1993,NaN,f,0-14,1,Andorra


In [436]:
tb_plus_country.drop(['country','country_name'], axis=1, inplace=True)
tb_plus_country

,Unnamed: 0,year,value,sex,age_group,country_id
0,0,1989,NaN,f,0-14,1
1,1,1990,NaN,f,0-14,1
2,2,1991,NaN,f,0-14,1
3,3,1992,NaN,f,0-14,1
4,4,1993,NaN,f,0-14,1
...,...,...,...,...,...,...
92299,92299,2004,NaN,m,NaN,208
92300,92300,2005,NaN,m,NaN,208
92301,92301,2006,NaN,m,NaN,208
92302,92302,2007,NaN,m,NaN,208


#### Save country_info to csv file

In [437]:
country_info.to_csv('country_info.csv', index=False)

#### Define function for converting values from a dictionary

In [438]:
def convert_x(x, dict):
    return dict[x]

#### Create sex_info dataset

ISO reference: https://en.wikipedia.org/wiki/ISO/IEC_5218<br>
WHO reference: https://apps.who.int/gho/data/node.searo-metadata.SEX?lang=en

In [439]:
tb_sdata = tb_plus_country[['year','value','country_id','sex','age_group']].copy()
tb_sdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92304 entries, 0 to 92303
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   year        92304 non-null  int64  
 1   value       34160 non-null  float64
 2   country_id  92304 non-null  int64  
 3   sex         92304 non-null  object 
 4   age_group   80766 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 4.2+ MB


In [440]:
tb_sdata.shape

(92304, 5)

In [441]:
tb_sdata.sex.value_counts()

m    46152
f    46152
Name: sex, dtype: int64

In [442]:
sex_info = tb_sdata.groupby(['sex']).size().reset_index(name='num_rows')
sex_info

,sex,num_rows
0,f,46152
1,m,46152


In [443]:
sex_info.drop(['num_rows'], axis=1, inplace=True)
sex_info

,sex
0,f
1,m


In [444]:
sex_name_dict = {"f":"FEMALE", "m":"MALE"}
sex_iso_dict = {"f":2, "m":1}
sex_who_dict = {"f":"FMLE", "m":"MLE"}

In [445]:
sex_info['sex_name'] = sex_info.apply(lambda x: convert_x(x['sex'],sex_name_dict),axis=1)
sex_info['sex_id'] = sex_info.apply(lambda x: convert_x(x['sex'],sex_iso_dict),axis=1)
sex_info['sex_who'] = sex_info.apply(lambda x: convert_x(x['sex'],sex_who_dict),axis=1)
sex_info.head()

,sex,sex_name,sex_id,sex_who
0,f,FEMALE,2,FMLE
1,m,MALE,1,MLE


In [446]:
sex_info = sex_info[['sex_id','sex_who','sex','sex_name']]
sex_info

,sex_id,sex_who,sex,sex_name
0,2,FMLE,f,FEMALE
1,1,MLE,m,MALE


In [447]:
tb_plus_sex = pd.merge(tb_sdata, 
                       sex_info,
                       how="left",
                       on="sex")
tb_plus_sex.head()

,year,value,country_id,sex,age_group,sex_id,sex_who,sex_name
0,1989,NaN,1,f,0-14,2,FMLE,FEMALE
1,1990,NaN,1,f,0-14,2,FMLE,FEMALE
2,1991,NaN,1,f,0-14,2,FMLE,FEMALE
3,1992,NaN,1,f,0-14,2,FMLE,FEMALE
4,1993,NaN,1,f,0-14,2,FMLE,FEMALE


In [448]:
tb_plus_sex.drop(['sex','sex_who','sex', 'sex_name'], axis=1, inplace=True)
tb_plus_sex

,year,value,country_id,age_group,sex_id
0,1989,NaN,1,0-14,2
1,1990,NaN,1,0-14,2
2,1991,NaN,1,0-14,2
3,1992,NaN,1,0-14,2
4,1993,NaN,1,0-14,2
...,...,...,...,...,...
92299,2004,NaN,208,NaN,1
92300,2005,NaN,208,NaN,1
92301,2006,NaN,208,NaN,1
92302,2007,NaN,208,NaN,1


#### Write sex_info to file

In [449]:
sex_info.to_csv('sex_info.csv', index=False)

#### Create age_group dataset

ISO reference: I could not find ISO standards for age groups.<br>
WHO reference: https://apps.who.int/gho/data/node.metadata.AGEGROUP?lang=en WHO had no 65+; they had 65-100 and 100+.

In [450]:
tb_adata = tb_plus_sex[['year','value','country_id','sex_id','age_group']].copy()
tb_adata.head()

,year,value,country_id,sex_id,age_group
0,1989,NaN,1,2,0-14
1,1990,NaN,1,2,0-14
2,1991,NaN,1,2,0-14
3,1992,NaN,1,2,0-14
4,1993,NaN,1,2,0-14


In [451]:
tb_adata.age_group.value_counts()

45-54    11538
65+      11538
15-24    11538
25-34    11538
0-14     11538
55-64    11538
35-44    11538
Name: age_group, dtype: int64

In [452]:
tb_adata['age_group'].isnull().sum()

11538

In [453]:
missing_age = tb_adata.loc[tb_adata['age_group'].isna(), :]
missing_age

,year,value,country_id,sex_id,age_group
40383,1989,NaN,1,2,NaN
40384,1990,NaN,1,2,NaN
40385,1991,NaN,1,2,NaN
40386,1992,NaN,1,2,NaN
40387,1993,NaN,1,2,NaN
...,...,...,...,...,...
92299,2004,NaN,208,1,NaN
92300,2005,NaN,208,1,NaN
92301,2006,NaN,208,1,NaN
92302,2007,NaN,208,1,NaN


#### Fill age_group NaN values with 'unknown'

In [454]:
tb_adata['age_group'] = tb_adata['age_group'].fillna('Unknown')
tb_adata

,year,value,country_id,sex_id,age_group
0,1989,NaN,1,2,0-14
1,1990,NaN,1,2,0-14
2,1991,NaN,1,2,0-14
3,1992,NaN,1,2,0-14
4,1993,NaN,1,2,0-14
...,...,...,...,...,...
92299,2004,NaN,208,1,Unknown
92300,2005,NaN,208,1,Unknown
92301,2006,NaN,208,1,Unknown
92302,2007,NaN,208,1,Unknown


In [455]:
tb_ages = tb_adata.groupby(['age_group']).size().reset_index(name='num_rows')
tb_ages

,age_group,num_rows
0,0-14,11538
1,15-24,11538
2,25-34,11538
3,35-44,11538
4,45-54,11538
5,55-64,11538
6,65+,11538
7,Unknown,11538


In [456]:
tb_ages.drop(['num_rows'], axis=1, inplace=True)
tb_ages

,age_group
0,0-14
1,15-24
2,25-34
3,35-44
4,45-54
5,55-64
6,65+
7,Unknown


In [457]:
tb_ages['age_group_id'] = tb_ages.index + 1
tb_ages

,age_group,age_group_id
0,0-14,1
1,15-24,2
2,25-34,3
3,35-44,4
4,45-54,5
5,55-64,6
6,65+,7
7,Unknown,8


In [458]:
age_group_info = tb_ages[['age_group_id','age_group']].copy()
age_group_info

,age_group_id,age_group
0,1,0-14
1,2,15-24
2,3,25-34
3,4,35-44
4,5,45-54
5,6,55-64
6,7,65+
7,8,Unknown


In [459]:
tb_plus_age = pd.merge(tb_adata, 
                       age_group_info,
                       how="left",
                       on="age_group")
tb_plus_age.head()

,year,value,country_id,sex_id,age_group,age_group_id
0,1989,NaN,1,2,0-14,1
1,1990,NaN,1,2,0-14,1
2,1991,NaN,1,2,0-14,1
3,1992,NaN,1,2,0-14,1
4,1993,NaN,1,2,0-14,1


In [460]:
tb_values = tb_plus_age[['year','country_id','sex_id','age_group_id','value']].copy()
tb_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92304 entries, 0 to 92303
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   year          92304 non-null  int64  
 1   country_id    92304 non-null  int64  
 2   sex_id        92304 non-null  int64  
 3   age_group_id  92304 non-null  int64  
 4   value         34160 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 4.2 MB


In [461]:
tb_values.head()

,year,country_id,sex_id,age_group_id,value
0,1989,1,2,1,NaN
1,1990,1,2,1,NaN
2,1991,1,2,1,NaN
3,1992,1,2,1,NaN
4,1993,1,2,1,NaN


In [462]:
tb_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92304 entries, 0 to 92303
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   year          92304 non-null  int64  
 1   country_id    92304 non-null  int64  
 2   sex_id        92304 non-null  int64  
 3   age_group_id  92304 non-null  int64  
 4   value         34160 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 4.2 MB


#### Write age_group_info and tb_values to files

In [463]:
age_group_info.to_csv('age_group_info.csv', index=False)

In [464]:
tb_values.rename(columns={"value": "count"}, inplace=True)
tb_values.head(10)

,year,country_id,sex_id,age_group_id,count
0,1989,1,2,1,NaN
1,1990,1,2,1,NaN
2,1991,1,2,1,NaN
3,1992,1,2,1,NaN
4,1993,1,2,1,NaN
5,1994,1,2,1,NaN
6,1996,1,2,1,0.0
7,1997,1,2,1,0.0
8,1998,1,2,1,NaN
9,1999,1,2,1,0.0


In [465]:
tb_values.to_csv('tb_counts.csv', index=False, na_rep='NULL')